In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import copy
import time

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from learning_wavelets.data import im_generators, ConcatenateGenerators
from learning_wavelets.keras_utils.thresholding import SoftThresholding, HardThresholding, DynamicSoftThresholding, DynamicHardThresholding
from learning_wavelets.learned_wavelet import learned_wavelet, learnlet
from learning_wavelets.unet import unet
from learning_wavelets.utils.metrics import metrics_for_params, metrics_exact_recon_net, metrics_original, metrics_wavelets, metrics_bm3d, metrics_dynamic_denoising_net


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.3.0     installed
numpy          : >=1.16.4  - required | 1.17.4    installed
matplotlib     : >=3.0.0   - required | 3.1.0     installed
astropy        : >=3.0.0   - required | 3.1.2     installed
nibabel        : >=2.3.2   - required | 2.5.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.4.1     installed
scikit-learn   : >=0.19.1  - requi

In [4]:
np.random.seed(0)

In [5]:
all_net_params = [
    {
        'name': 'unet',
        'init_function': unet,
        'run_params': {
            'n_layers': 5, 
            'pool': 'max', 
            "layers_n_channels": [64, 128, 256, 512, 1024], 
            'layers_n_non_lins': 2,
            'non_relu_contract': False,
            'input_size': (None, None, 1),
        },
        'run_id': 'unet_div2k_30_1574408775',
    },
    {
        'name': 'learned wavelet_st',
        'init_function': learned_wavelet,
        'run_params': {
            'n_scales': 5, 
            'n_details': 256, 
            'n_coarse': 1, 
            'mixing_details': False,
            'denoising_activation': SoftThresholding,
            'wav_pooling': True,
            'wav_use_bias': False,
            'wav_normed': True,
            'filters_normed': ['details', 'coarse'],
            'input_size': (None, None, 1),     
        },
        'run_id': 'learned_wavelet_div2k_30_1574790444',
        'epoch': 250,
    },
]
all_exact_recon_net_params = [
    {
        'name': 'learnlet_exact_reco',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': SoftThresholding,
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs':{
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 1,
            'input_size': (None, None, 1),     
        },
        'run_id': 'learnlet_div2k_30_1575052510',
        'epoch': 250,
    },
]

dynamic_denoising_net_params = [
    {
        'name': 'learnlet_0_55',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=False),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (None, None, 1),     
        },
        'run_id': 'learnlet_dynamic_st_div2k_(0, 55)_1575835369',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_alpha_learned',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=True),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (None, None, 1),     
        },
        'run_id': 'learnlet_dynamic_st_div2k_(0, 55)_1575881431',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_alpha_learned_bsd',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=True),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (size, size, n_channels),     
        },
        # there was a typo during training, but it is actually st
        'run_id': 'learnlet_dynamic_ht_bsd500_0_55_1575925545',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_ht',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicHardThresholding(3, trainable=False),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (None, None, 1),     
        },
        'run_id': 'learnlet_dynamic_ht_div2k_(0, 55)_1575898840',
        'epoch': 250,
    },
]

In [6]:
noise_stds = [5, 15, 25, 30, 50, 75]

In [7]:
noise_std_metrics = {}
for noise_std in tqdm_notebook(noise_stds, 'Noise stds'):
    n_channels = 1
    im_gen_train, im_gen_val, im_gen_test, size, n_samples_train = im_generators(
        'bsd68', 
        batch_size=1, 
        validation_split=0, 
        no_augment=True, 
        noise_std=noise_std,
        grey=True
    )    
    im_bsd68 = im_gen_test
    metrics = []
    for net_params in all_net_params:
        at_inference_net_params = copy.deepcopy(net_params)
        if callable(at_inference_net_params['run_params'].get('denoising_activation')):
            at_inference_net_params['run_params']['denoising_activation'] = at_inference_net_params['run_params']['denoising_activation'](2*noise_std/255)
        metrics.append((net_params['name'], metrics_for_params(im_gen_test, **at_inference_net_params)))
        
    for net_params in dynamic_denoising_net_params:
        metrics.append((net_params['name'], metrics_dynamic_denoising_net(im_gen_test, noise_std, **net_params)))

    for net_params in all_exact_recon_net_params:
        at_inference_net_params = copy.deepcopy(net_params)
        if callable(at_inference_net_params['run_params'].get('denoising_activation')):
            at_inference_net_params['run_params']['denoising_activation'] = at_inference_net_params['run_params']['denoising_activation'](2*noise_std/255)
        metrics.append((net_params['name'], metrics_exact_recon_net(im_gen_test, **at_inference_net_params)))

    metrics.append(('original', metrics_original(im_gen_test)))
    # metrics.append(('bm3d', metrics_bm3d(im_gen_test)))
    metrics.append(('wavelets_24', metrics_wavelets(im_gen_test, '24', noise_std=noise_std)))
    metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())
    noise_std_metrics[noise_std] = metrics

In [8]:
noise_std_metrics

{5: [('unet', PSNR = 27.6 +/- 5.228 SSIM = 0.7227 +/- 0.1863),
  ('learned wavelet_st', PSNR = 30.39 +/- 6.608 SSIM = 0.9287 +/- 0.03775),
  ('original', PSNR = 34.15 +/- 0.03261 SSIM = 0.8854 +/- 0.1022),
  ('learnlet_exact_reco', PSNR = 35.03 +/- 3.761 SSIM = 0.9453 +/- 0.03728),
  ('learnlet_0_55', PSNR = 35.35 +/- 3.86 SSIM = 0.9471 +/- 0.04034),
  ('learnlet_0_55_alpha_learned',
   PSNR = 35.4 +/- 4.039 SSIM = 0.9477 +/- 0.0408),
  ('wavelets_24', PSNR = 35.69 +/- 3.741 SSIM = 0.9449 +/- 0.04547),
  ('learnlet_0_55_ht', PSNR = 35.78 +/- 3.452 SSIM = 0.9509 +/- 0.03361)],
 15: [('original', PSNR = 24.61 +/- 0.03261 SSIM = 0.5689 +/- 0.2587),
  ('unet', PSNR = 27.81 +/- 5.091 SSIM = 0.7337 +/- 0.1797),
  ('wavelets_24', PSNR = 29.41 +/- 4.875 SSIM = 0.8205 +/- 0.1246),
  ('learnlet_exact_reco', PSNR = 30.11 +/- 4.604 SSIM = 0.8413 +/- 0.1024),
  ('learned wavelet_st', PSNR = 30.15 +/- 4.721 SSIM = 0.8476 +/- 0.09783),
  ('learnlet_0_55_ht', PSNR = 30.16 +/- 4.252 SSIM = 0.8395 +/- 0

In [9]:
# PSNR table
psnr_metrics_table = pd.DataFrame(
    index=noise_stds, 
    columns=[p['name'] for p in all_net_params] + [p['name'] for p in all_exact_recon_net_params] + [p['name'] for p in dynamic_denoising_net_params] + ['original'],
)
for noise_std, metrics in noise_std_metrics.items():
    for name, m in metrics:
        psnr_metrics_table.loc[noise_std, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['PSNR'].mean(), 
            std=m.metrics['PSNR'].stddev(),
        )
psnr_metrics_table

,unet,learned wavelet_st,learnlet_exact_reco,learnlet_0_55,learnlet_0_55_alpha_learned,learnlet_0_55_ht,original,wavelets_24
5,27.6 (2.614),30.39 (3.304),35.03 (1.881),35.35 (1.93),35.4 (2.02),35.78 (1.726),34.15 (0.01631),35.69 (1.87)
15,27.81 (2.545),30.15 (2.36),30.11 (2.302),30.25 (2.257),30.32 (2.255),30.16 (2.126),24.61 (0.01631),29.41 (2.437)
25,28.26 (2.486),28.08 (2.312),27.87 (2.353),27.91 (2.336),27.98 (2.312),27.71 (2.271),20.17 (0.01631),27.08 (2.497)
30,28.25 (2.61),27.26 (2.301),27.08 (2.343),27.1 (2.342),27.17 (2.31),26.9 (2.306),18.59 (0.01631),26.32 (2.488)
50,18.49 (0.4162),25.0 (2.207),24.9 (2.24),24.95 (2.287),24.97 (2.228),24.79 (2.325),14.15 (0.01631),24.42 (2.41)
75,13.51 (0.1955),23.25 (2.045),23.2 (2.074),23.32 (2.154),23.28 (2.069),23.26 (2.241),10.63 (0.01631),23.06 (2.307)


In [10]:
# SSIM table
ssim_metrics_table = pd.DataFrame(
    index=noise_stds, 
    columns=[p['name'] for p in all_net_params] + [p['name'] for p in all_exact_recon_net_params] + [p['name'] for p in dynamic_denoising_net_params] + ['original', 'wavelets_24'],
)
for noise_std, metrics in noise_std_metrics.items():
    for name, m in metrics:
        ssim_metrics_table.loc[noise_std, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['SSIM'].mean(), 
            std=m.metrics['SSIM'].stddev(),
        )
ssim_metrics_table

,unet,learned wavelet_st,learnlet_exact_reco,learnlet_0_55,learnlet_0_55_alpha_learned,learnlet_0_55_ht,original,wavelets_24
5,0.7227 (0.09316),0.9287 (0.01887),0.9453 (0.01864),0.9471 (0.02017),0.9477 (0.0204),0.9509 (0.01681),0.8854 (0.05111),0.9449 (0.02273)
15,0.7337 (0.08987),0.8476 (0.04891),0.8413 (0.05119),0.8414 (0.05274),0.8437 (0.05159),0.8395 (0.05276),0.5689 (0.1293),0.8205 (0.06228)
25,0.7631 (0.08133),0.7615 (0.06025),0.7542 (0.06246),0.7544 (0.06454),0.7567 (0.06221),0.7488 (0.0659),0.3884 (0.1283),0.7321 (0.07773)
30,0.7802 (0.06849),0.7231 (0.06312),0.7159 (0.06543),0.7171 (0.06794),0.7189 (0.0649),0.7112 (0.07007),0.328 (0.1201),0.6974 (0.08341)
50,0.2841 (0.09905),0.5962 (0.06742),0.5899 (0.06972),0.5969 (0.074),0.595 (0.06838),0.5945 (0.08054),0.1832 (0.08348),0.5955 (0.09832)
75,0.1439 (0.06663),0.4806 (0.06402),0.4741 (0.06546),0.4875 (0.0717),0.4811 (0.0643),0.4945 (0.0833),0.1013 (0.05182),0.5141 (0.1057)
